In [1]:
import pandas as pd
import warnings
from sqlalchemy import create_engine, MetaData, Table, select, and_, extract, func, distinct
from datetime import datetime, timedelta, date
from pandas.tseries.offsets import Week, Day
from dotenv import load_dotenv
import os

### CREACION DE LA FUNCION DE SELECCION DE FECHAS ###

def select_date(ref_week, year):
    """
    Regresa la marca de tiempo de inicio y de fin de cada semana para la operacion del historial tecnico,
    con las siguientes consideraciones: una semana de de ancho entre el inicio y fin,  los inicios inician el domingo y
    terminan el sabado, la semana 1 y semamana 53 son irregulares por lo que terminan de acuerdo al año calendario.

    Parámetros:
    ref_week (float): numero de semana en curso.
    año (float): año de semana en curso.

    Retorna:
    tupla(Timestamp, Timestamp, Timestamp): fecha de inicio, fecha de fin de acuerdo al año y semana de referencia, 
    Fecha de inicial.
    """

    init_date = pd.Timestamp(year=year, month=1, day=1, hour=0, minute=0, second= 0)
    if ref_week == 1: # Semana 1, se cruza entre el año anterior y el actual, semana irregular
        cum_offset = pd.offsets.Week(weekday=init_date.weekday(), n=ref_week-1)
        btw_offset = pd.offsets.Week(weekday=5)
    elif ref_week == 53: # Semana 53, se cruza entre entre el año actual y el siguiente, semana irregular
        cum_offset = pd.offsets.Week(weekday=6, n=ref_week-1)
        btw_offset = pd.offsets.Week(weekday=(init_date + pd.offsets.YearEnd()).weekday()) 
    elif ref_week > 53:
        raise ValueError('Solo existen 53 semanas maximo por año, favor de revisar')       
    else:
        cum_offset = pd.offsets.Week(weekday=6, n=ref_week-1)
        btw_offset = pd.offsets.Week(weekday=5)
    
    adjust_hms = timedelta(hours=23, minutes=59, seconds=59) 
    # cum_offset es el numero de semanas acumuladas y desplazadas
    # btw_offset es el numero de semanas desplazadas entre el time_start y time_end
    # adjust_hms representa el desplazamiento de tiempo para el cierre del último día
    
    time_start = init_date + cum_offset
    time_end = init_date + cum_offset +  btw_offset + adjust_hms 

    return time_start, time_end, init_date

time_start, time_end, init_date = select_date(1, 2024)

### IMPORTACION DE COORDENADAS DE GEOCERCAS DE LA BBDD A DF ###

# Cargar el archivo .env
load_dotenv(dotenv_path='data/.env')


# Parámetros de conexión a la base de datos SQL Server
dbuser = os.getenv('DBUSER')
password = os.getenv('PASSWORD')
server = os.getenv('SERVER')
port = os.getenv('PORT')
database1 = os.getenv('DATABASE1')
database2 = os.getenv('DATABASE2')
driver = os.getenv('DRIVER')


# Cadenas de conexión
connection_url1 = f'mssql+pyodbc://{dbuser}:{password}@{server}:{port}/{database1}?driver={driver}'
connection_url2 = f'mssql+pyodbc://{dbuser}:{password}@{server}:{port}/{database2}?driver={driver}'


# Crear engines
engine1 = create_engine(connection_url1)
engine2 = create_engine(connection_url2)


# Crear varias instancias de MetaData y reflejar las tablas desde la base de datos
metadata1 = MetaData()
metadata2 = MetaData()

metadata1.reflect(bind=engine1)
metadata2.reflect(bind=engine2)



# Seleccionar todas las filas de sus respectivas tablas
HISTORIALTECNICO_FINAL = metadata1.tables['HISTORIALTECNICO_FINAL']
AREAS_TBJ = metadata2.tables['AREAS_TBJ']


# Construir los 'select' correspondientes


not_economico = pd.read_csv('data/not_economico.csv').values.tolist()


select1 = select(HISTORIALTECNICO_FINAL.c.Eco, 
                       HISTORIALTECNICO_FINAL.c.FechaI,
                       HISTORIALTECNICO_FINAL.c.Rancho,
                       HISTORIALTECNICO_FINAL.c.Tabla,
                       HISTORIALTECNICO_FINAL.c.Longitud,
                       HISTORIALTECNICO_FINAL.c.Latitud).where(
    HISTORIALTECNICO_FINAL.c.FechaI.between(time_start, time_end),
    HISTORIALTECNICO_FINAL.c.Eco.notin_(not_economico) 
)


select2 = select(distinct(AREAS_TBJ.c.Fecha))


# Conectar los 'select()' y 'engine' y obtener los datos los DataFrames correspondientes

df1 = pd.read_sql(select1, engine1)
df2 = pd.read_sql(select2, engine2)



In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point,LineString , Polygon
import shapely.geometry as geom

### AGRUPACION DE COORDENADAS DE GEOCERCAS  ###


# Crear una geometría Point para cada fila del DataFrame
geometry = [Point(xy) for xy in zip(df1['Longitud'], df1['Latitud'])]

# Crear un GeoDataFrame y asignar la columna de geometría
gdf = gpd.GeoDataFrame(df1, geometry=geometry, crs="EPSG:4326")

# Crear la columna fecha sin fecha y hora
gdf["Fecha"] = gdf["FechaI"].dt.date


dissolved = gdf.dissolve(by=['Fecha','Eco','Tabla']).reset_index()
# sjoined = dissolved.sjoin(catalog,  how='left', predicate='within')
# dissolved

In [3]:
### CATALOGO CON COORDENADAS Y HECTÁREAS ###

from shapely import wkt


namelist = {'Hectareas': 'HasTot', 'Poligono': 'CoordsTabla', 'Nombre HT': 'Tabla', 'Rancho HT': 'Rancho'}


catalog = pd.read_excel('data/CATALOGO DE GEOCERCAS.xlsx').dropna(subset=['Poligono'])
catalog = catalog.rename(columns=namelist)
catalog = catalog.query("Categoria == '03 Tabla'")
catalog['CoordsTabla'] = catalog['CoordsTabla'].apply(wkt.loads)
catalog = gpd.GeoDataFrame(catalog, geometry='CoordsTabla', crs='EPSG:4326')

### UNION ENTRE GEOCERCAR AGRUPADAS Y CATALOGO DE COORDENADAS ###

merged = dissolved.merge(catalog,  how='left', left_on=['Tabla', 'Rancho'], right_on=['Tabla', 'Rancho'], validate = 'many_to_one').set_index(['Fecha','Eco','Tabla'])
# merged

In [4]:
### CONVERTIR LOS PUNTOS AGRUPADOS EN OTRAS GEOMETRIAS###


from alphashape import alphashape
import numpy as np
from shapely.geometry import MultiPoint


def generate_geom(x):
    if x['geometry'].geom_type == 'Point':
        return x['geometry']
    elif x['geometry'].geom_type != 'Point':    
        if x.name[2] == '-':
            return LineString(x['geometry'].geoms)
        elif x.name[2] != '-':
            return alphashape(x['geometry'].geoms, 0)

merged['gengeom'] = merged.apply(generate_geom, axis=1)
merged['gengeom'].crs = 'EPSG:4326'

### OBTENER LAS HECTAREAS TRABAJADAS ###

merged['HasTbj'] = round(merged['gengeom'].to_crs('EPSG:32614').area/10000, 2)
# merged

In [5]:
### DAR EL FORMATO A LA TABLA PARA LA SALIDA ###

order = ['Eco','Fecha', 'Codigo ERP','Tabla', 'HasTbj','HasTot','gengeom',
         'UrlImagen', 'LaborUsoMaqID', 'ImplementoUsoMaqID','Activo']

namelist = {'gengeom':'Geometria', 'Codigo ERP':'Codigo'}


merged_queried = merged[(merged['gengeom'].geom_type == 'Polygon')&(merged['HasTot'].notna())].reset_index()
merged_queried = merged_queried.reindex(columns=order).rename(columns=namelist)
merged_queried.index = merged_queried.index.rename('HistorialID')

merged_queried['Fecha'] = pd.to_datetime(merged_queried['Fecha'])
first_date_df =  merged_queried['Fecha'].min() 
merged_queried['Fecha'].unique()

<DatetimeArray>
['2024-01-01 00:00:00', '2024-01-02 00:00:00', '2024-01-03 00:00:00',
 '2024-01-04 00:00:00', '2024-01-05 00:00:00', '2024-01-06 00:00:00']
Length: 6, dtype: datetime64[ns]

In [6]:
### CREACIÓN DE UNA FUNCION DE VALIDACION (PARA INTEGRIDAD DE DATOS) ###

import warnings


df_dates =  pd.Series(pd.to_datetime(merged.index.get_level_values('Fecha').unique())) # Representan las fechas de las geometrias que van a ingresar a la BBDD 
# df_dates =  merged_queried['Fecha'].unique()
bd_dates =  df2['Fecha'] # Representan las fechas de las geometrias que ya ingresaron a la BBDD

unstb_date = pd.Timestamp.now().date() - (pd.offsets.Week(weekday=6) * 3) # Es la fecha/marca de tiempo a partir de donde ya se considera 'demasiado reciente' y los datos no son tan confiables todavia
all_dates = pd.concat([bd_dates, df_dates]) # Representan una combinacion de las dos fechas: [no han ingresado] + [van a ingresar]  
full_dates = pd.Series(pd.date_range(start=init_date, end=time_end, freq='D')) # Es un RANGO DE FECHAS GENERADO la fecha minima de la BBDD (si está posible, si no elige otra) y la fecha final seleccionada (NO DE LA BBDD)



def validate_entry():  
    
    """
    Valida que las fechas ingresadas no estén en la base datos ANTES de que ingresen, evitando que la agregues,
    si la fecha es demasiado reciente (cualquier superior a la productiva y dos hasta dos semanas anteriores),
    o que alguno de los días de la semana productiva ya se encuentre en la base de datos,

    Por otro lado también da el aviso si hay días no incluidos en la base de datos, los cuales se deben evaluar,
    por ejemplo si es un día que no hubo operacion es normal que aparezca el aviso, si son varios y no hay razón es 
    posible que se haya saltado una semana la ejecucion del programa.

    Parámetros:
    None

    Retorna:
    None
    """


    chklmt_df = any(df_dates >= unstb_date) # Retorna solo 'True' si existe AL MENOS una fecha que IGUALE O SUPERE a la fecha de referencia (EL LIMITE DONDE LAS FECHAS SE CONSIDERAN ESTABLES), de lo contrario retorna 'False'
    bd_empty = bd_dates.empty # Retorna 'True' si la BBDD no contiene ningun registro, de lo contrario retorna 'False'
    
    chkdup_bd = any(all_dates.duplicated()) # Retorna 'True' si existe AL MENOS un duplicado, de lo contrario retorna 'False' (es decir no debe exitir ningún duplicado para que aparezca 'False')
    chk_init =  any(df_dates.isin([init_date])) # Revisa que las fechas que van a ingresar a la BBDD tengan la fecha inicial (la primer fecha que de debería estar en la BBDD) 
    chk_miss  = all(full_dates.isin(all_dates)) # Retorna 'True' si al menos una fecha full_dates NO SE ENCUENTRA dentro de all_dates (la idea evitar posibles gaps  de fechas faltantes comparando, all_dates (fechas reales) con full_dates (fechas generadas))
    dts_miss = np.array(full_dates[~full_dates.isin(all_dates)].dt.date)
    if chklmt_df: 
        raise RuntimeError('Error: Has agregado una fecha demasiado reciente')
   
    elif bd_empty:
         if not chk_init:
            warnings.warn(f'No se encontró el {init_date} (fecha de inicio) en las fechas de hectareas, si la incluiste en tu primer selección ignora este mensaje, de lo contrario revisa tu selección y agregalá a la BBDD', RuntimeWarning)
            # raise RuntimeError('Error: Debes agregar una fecha que contenga de inicio de año en una BBDD vacia')
            
    elif not bd_empty:
        if chkdup_bd:
            raise RuntimeError('Error: Alguna fecha que estas intentando agregar ya esta en la BBDD')
        
        if not chk_miss:
            warnings.warn(f'Las fechas {dts_miss} no están presentes, si las incluiste pero fueron filtradas en el cálculo ignora este mensaje, pero si no se seleccionó, debes generar las fechas faltantes', RuntimeWarning)
            # raise RuntimeError('Error: Hay fechas faltantes, haz de nuevo la seleccion con fechas continuas')
            

In [38]:

### DAR SALIDA A LOS DATOS EN SQL O EN EXCEL ###


# validate_entry()

try:
    validate_entry()

except RuntimeError as err:
    print(err)

else:
    merged_queried.to_wkt().to_sql('AREAS_TBJ', con=engine2, index=False, if_exists='append')
    bd_dates = pd.read_sql(select2, engine2)['Fecha']
    all_dates = pd.concat([bd_dates, df_dates])

# merged_queried.to_excel('output.xlsx', index=True)
